## Setup
Initialize connection to the hubs

In [1]:
from pybricksdev.connections import BLEPUPConnection
from asyncio import gather, sleep

switch = BLEPUPConnection()
await switch.connect('90:84:2B:01:14:27')
    
train = BLEPUPConnection()
await train.connect('90:84:2B:04:C3:81')

Connecting to 90:84:2B:01:14:27
Connected successfully!
Connecting to 90:84:2B:04:C3:81
Connected successfully!


## Train script
Output color sensor detection to variable

In [2]:
%%file build/train.py

from pybricks.hubs import CityHub
from pybricks.pupdevices import ColorDistanceSensor
from pybricks.parameters import Port, Color
from pybricks.tools import wait

hub = CityHub()
sensor = ColorDistanceSensor(Port.B)
speed = 0

#wait(2000)

while True:
    color = sensor.color()
    #print(color)
    if color == Color.RED:
        hub.light.on(Color.RED)
        speed = 0
    elif color == Color.YELLOW:
        hub.light.on(Color.YELLOW)
        speed = 1
    elif color == Color.WHITE:
        hub.light.on(Color.WHITE)
        speed = 2
    print(speed)
    #print("__data__{0}".format(speed))
    wait(500)

Overwriting build/train.py


## Switch script
Evaluate value recieved from train and act on it

In [3]:
%%file build/switch.py

from pybricks.hubs import CityHub
from pybricks.pupdevices import ColorDistanceSensor, DCMotor
from pybricks.parameters import Port, Color
from pybricks.tools import wait
from pybricks.experimental import getchar

hub = CityHub()
motor = DCMotor(Port.A)
speed = 0

wait(2000)

while True:
    #motor.stop()
    try:
        #speed = eval(input())
        speed = getchar()
    except:
        print(speed)
        print("switch: didn't get value")
        pass
    else:
        #if speed is not None:
        if speed == 0:
            hub.light.on(Color.RED)
        elif speed == 1:
            hub.light.on(Color.YELLOW)
            motor.dc(100)
            wait(200)
            motor.stop()
        elif speed == 2:
            hub.light.on(Color.WHITE)
            motor.dc(-100)
            wait(400)
            motor.stop()
    wait(500)

Overwriting build/switch.py


## PC script
Reads value from Train and sends it to all other hubs

In [4]:
# We can do anything we want with the data, like displaying it.
# In this example, we just send the received number from the
# remote to the car as a single byte

#DATA = b'__data__'

async def forwarder(switch, train):
    
    # Give the hubs some time to start
    await sleep(5)

    while switch.state == switch.RUNNING or train.state == train.RUNNING:
        # Check if the remote has printed anything
        if len(train.output) > 0:
            
            # If so, let's see what the most recent value is
            line = train.output[-1]
            
            # In this demo, we are only interested in the most recent value,
            # so we can clear everything else.
            train.output = []
            
            # Try to convert it to a number and send it as one byte to the receiving hub
            try:
                speed = int(line.decode())
                print ("pc: ", speed)
                await switch.write(bytes([speed]))
                #await switch.write(line[len(DATA):] + b'\r\n')
            except:
                print("skipped")
                pass
        await sleep(0.5)

## Run everything

In [ ]:
await gather(
    switch.run('build/switch.py'),
    train.run('build/train.py', print_output=False),
    #switch.run('build/switch.py', print_output=False),
    #train.run('build/train.py'),
    forwarder(switch, train)
)


pc:  2
pc:  2
pc:  2
pc:  2
pc:  2
pc:  2
pc:  2
pc:  2
pc:  2


In [1]:
# Disconnect (optional)

await switch.disconnect()
await train.disconnect()

NameError: name 'switch' is not defined